In [2]:
# STANDARD
from pprint import pprint

# PyPI
from bs4 import BeautifulSoup as BS
import requests
from tqdm import tqdm

# LOCAL
from interface_db import db_interface_sqlite as data
from interface_website import website_tools as tools
from interface_website import website_patheos as patheos

In [2]:
with data.database() as db:
    db.create_tables()
    db.insert_update_site_pages(0, 149)
    result = db.execute(f"SELECT number FROM site_pages WHERE site_id = 47")
    #print(result[0][0])
    for i in result:
        results = patheos.fetch_and_insert_blogs(i[0])
        print(f'Inserted: {results.inserted} | Not Inserted: {results.not_inserted} | Errors: {results.exceptions}')

In [3]:
patheos.insert_website('Patheos Blogs', 'https://patheos.com/blogs')
results = patheos.fetch_and_insert_categories('Patheos Blogs', 1)
with data.database() as db:
    result = db.execute("SELECT name FROM categories")
    for i in result:
        results = patheos.fetch_and_insert_blogs(i[0])
        print(f'Inserted: {results.inserted} | Not Inserted: {results.not_inserted} | Errors: {results.exceptions}')

Inserting Patheos Blogs - https://patheos.com/blogs
Not inserted. If exists, then https://patheos.com/blogs is already present, named Patheos Blogs.
Pulling categories for Patheos Blogs
Inserted: 5 | Not Inserted: 0 | Errors: 0
Inserted: 89 | Not Inserted: 0 | Errors: 0
Inserted: 8 | Not Inserted: 0 | Errors: 0
Inserted: 84 | Not Inserted: 0 | Errors: 0
Inserted: 18 | Not Inserted: 0 | Errors: 0
Inserted: 4 | Not Inserted: 0 | Errors: 0
Inserted: 6 | Not Inserted: 0 | Errors: 0
Inserted: 17 | Not Inserted: 0 | Errors: 0
Inserted: 34 | Not Inserted: 0 | Errors: 0
Inserted: 32 | Not Inserted: 3 | Errors: 0
Inserted: 65 | Not Inserted: 0 | Errors: 0
Inserted: 59 | Not Inserted: 0 | Errors: 0
Inserted: 105 | Not Inserted: 1 | Errors: 0
Inserted: 2 | Not Inserted: 1 | Errors: 0
Inserted: 24 | Not Inserted: 0 | Errors: 0
Inserted: 3 | Not Inserted: 0 | Errors: 0
Inserted: 10 | Not Inserted: 0 | Errors: 0
Inserted: 12 | Not Inserted: 0 | Errors: 0
Inserted: 16 | Not Inserted: 0 | Errors: 0
In

In [5]:
import sys

number_of_new_pages = patheos.number_of_blog_pages('Jesus Creed')
print(f'\nNew Pages: {number_of_new_pages}')

1640
New Pages: 1639


In [3]:
import sys
import time
import datetime

results = tools.insert_results()


# PLACEHOLDER
def scrape_blog_initialize(website_id):
    with data.database() as db:
        website = db.query_websites(website_id=website_id)
    

# PLACEHOLDER
def script_category_initialize(category_id):
    with data.database() as db:
        category = db.query_categories(category_id=category_id)
        blog_list = db.execute(f"""SELECT id FROM categories WHERE category_id = {category.id}""")
    scrape_blog_initialize(blog_id=blog.id)

        
def scrape_blog_initialize(blog_id):
    with data.database() as db:
        blog = db.query_blogs(blog_id=blog_id)
        total_blogs = db.execute(f"""SELECT COUNT(*) FROM blogs WHERE category_id = {blog.category_id}""")[0][0]
    total_pages = patheos.number_of_blog_pages(blog_id=blog.id)
    resume_page = patheos.find_page_resume_scrape(blog.id)
    
    page_number = resume_page
    total = 0
    continue_on = True
    while continue_on == True:
        base_url = blog.url + '/page/' + str(page_number)
        #print(base_url)
        results_l = patheos.scrape_posts_on_page(base_url)
        if results_l != 404:
            for i in results_l:
                post = data.post(i, blog.id)
                post = patheos.scrape_post(post, 'no')
                if post.title == None:
                    print('no title')
                if tools.check_url_new('posts', post.url) == True:
                    with data.database() as db:
                        db.insert_post(post)
                        db.commit()
                        results.inserted += 1
                else:
                    # <placeholder for logging>
                    results.not_inserted += 1
                total += 1
                sys.stdout.write('\r' + 'BLOG: (' + str(blog.id) + '/' + str(total_blogs) + ') '  + blog.name 
                                      + ' | PAGE: ' + str(page_number) + '/' + str(total_pages) 
                                      + ' | TOTAL PROCESSED: ' + str(total) 
                                      + ' | INSERTED: ' + str(results.inserted)
                                      + ' | NOT INSERTED: ' + str(results.not_inserted))
        else:
            continue_on = False
        page_number += 1
        sys.stdout.write('\r' + 'BLOG: (' + str(blog.id) + '/' + str(total_blogs) + ') '  + blog.name 
                              + ' | PAGE: ' + str(page_number) + '/' + str(total_pages) 
                              + ' | TOTAL PROCESSED: ' + str(total) 
                              + ' | INSERTED: ' + str(results.inserted)
                              + ' | NOT INSERTED: ' + str(results.not_inserted))
    with data.database() as db:
        db.update_date_blog(blog)

In [4]:
with data.database() as db:
    list_websites = db.execute("""SELECT id FROM websites""")[0]
    #print(list_websites)
    for website_id in list_websites:
        website = db.query_websites(website_id=website_id)
        list_categories = db.execute(f"""SELECT id FROM categories WHERE website_id = {website.id}""")
        #print(list_categories)
        for category_id in list_categories:
            category = db.query_categories(category_id=category_id[0])
            print(category.id, category.name)
            list_blogs = db.execute(f"""SELECT id, name FROM blogs WHERE category_id={category.id} ORDER BY last_date""")
            for blog_id in list_blogs:
                blog = db.query_blogs(blog_id=blog_id[0])
                scrape_blog_initialize(blog.id)    

1 buddhist-blogs
BLOG: (5/5) Wild Fox Zen | PAGE: 2/85 | TOTAL PROCESSED: 25 | INSERTED: 974 | NOT INSERTED: 34RTED: 24

AttributeError: 'NoneType' object has no attribute 'text'